<a href="https://colab.research.google.com/github/PujanPandey07/Movie-Recommender-system-using-NLP-and-ML/blob/main/vectorembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from sklearn.preprocessing import MultiLabelBinarizer

## first importing the dataset file

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movies_df_processed.csv to movies_df_processed.csv


In [ ]:
final_set=pd.read_csv('movies_df_processed.csv')

## Now first let's compute vector embeddings for overview columns in our data

In [ ]:
model=SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_plot_embedding(text):
    if isinstance(text, str) and text.strip():
        embedding = model.encode(text, convert_to_tensor=True)
        return embedding.cpu().numpy()
    else:
        return None

In [ ]:

final_set['plot_vector'] = final_set['overview'].apply(get_plot_embedding)


## now lets get embeddings for generes

In [ ]:
def extract_genre_names(genre_str):
    try:
        genre_list = json.loads(genre_str)
        return [genre['name'] for genre in genre_list]
    except:
        return []

# Create a new column of genre lists
final_set['genre_list'] = final_set['genres'].apply(extract_genre_names)


In [ ]:
final_set['genre_text'] = final_set['genre_list'].apply(lambda x: " ".join(x))


In [ ]:
genre_embeddings = model.encode(final_set['genre_text'].tolist(), convert_to_tensor=False)

# Add as a new column
final_set['genre_vector'] = list(genre_embeddings)

## now we compute meta data embeddings and add a column for metadata embedding vector

In [ ]:
final_set['metadata_text'] = final_set['metadata_text'].fillna("")

In [ ]:
vectorizer = TfidfVectorizer()
metadata_tfidf = vectorizer.fit_transform(final_set['metadata_text'])
metadata_vectors = metadata_tfidf.toarray()

final_set['metadata_vector']=list(metadata_vectors)

## cleaning reviews so that we can compute their embedding

In [ ]:
import re
import string

def clean_review(text):
    if not isinstance(text, str):
        return ""

    # Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

## Computing vector embeddings of reviews

In [ ]:
import ast

def compute_avg_review_embedding(reviews, dim=384):
    # Convert stringified list to actual list if needed
    if isinstance(reviews, str):
        try:
            reviews = ast.literal_eval(reviews)
        except Exception:
            reviews = []
    if not isinstance(reviews, list):
        return np.zeros(dim)

    cleaned_reviews = [
        review.strip()
        for review in reviews
        if isinstance(review, str) and len(review.strip().split()) > 3
    ]

    if len(cleaned_reviews) == 0:
        return np.zeros(dim)

    try:
        embeddings = model.encode(cleaned_reviews, convert_to_tensor=False)
        avg_embedding = np.mean(embeddings, axis=0)
        return avg_embedding
    except Exception as e:
        print(f"⚠️ SBERT encoding failed: {e}")
        return np.zeros(dim)

final_set['review_vector'] = final_set['reviews'].apply(compute_avg_review_embedding)

In [ ]:
final_set['reviews']

,reviews
0,['Avatar is an overwhelming immersive spectacl...
1,['Part 3 Jack is stuck in Davy Jones locker. W...
2,['a hrefhttpwww.cutprintfilm.comreviewsspectre...
3,['I felt like this was a tremendous end to Nol...
4,['Totally forgettable and full of stereotypes'...
...,...
4794,['Filmed in 1992 and being Robert Rodriquezs f...
4795,No reviews available
4796,No reviews available
4797,No reviews available


## Now we combine all reviews into a long string so that we can find tfidf vectors for sentiment analysis

In [ ]:
import ast

def parse_reviews(cell):
    try:
        if isinstance(cell, str):
            return ast.literal_eval(cell)
        elif isinstance(cell, list):
            return cell
        else:
            return []
    except Exception:
        return []

# Apply to the column
final_set["parsed_reviews"] = final_set["reviews"].apply(parse_reviews)


In [ ]:
def combine_reviews(review_list):
    return ' '.join([r.strip() for r in review_list if isinstance(r, str) and r.strip()])

final_set["combined_review_text"] = final_set["parsed_reviews"].apply(combine_reviews)


In [ ]:

final_set["combined_review_text"]

,combined_review_text
0,Avatar is an overwhelming immersive spectacle....
1,Part 3 Jack is stuck in Davy Jones locker. Whi...
2,a hrefhttpwww.cutprintfilm.comreviewsspectrea\...
3,I felt like this was a tremendous end to Nolan...
4,Totally forgettable and full of stereotypes Ed...
...,...
4794,Filmed in 1992 and being Robert Rodriquezs fil...
4795,
4796,
4797,


In [ ]:
final_set["cleaned_reviews"]=final_set["combined_review_text"].apply(clean_review)


## Now saving our final dataset with embeddings as pickel files so we can use them later

In [ ]:
finaldataset=final_set[['id','title_x','keywords','overview','plot_vector','genre_vector','metadata_vector','review_vector','cleaned_reviews']]

In [ ]:
finaldataset.head()

,id,title_x,keywords,overview,plot_vector,genre_vector,metadata_vector,review_vector,cleaned_reviews
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[-0.006365536, 0.061562583, 0.030460315, -0.04...","[0.004819642, -0.011214975, -0.0059656696, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.047655895, -0.0069539687, 0.045017313, -0....",avatar is an overwhelming immersive spectacle ...
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[-0.040365733, -0.027182437, -0.085236676, -0....","[0.006769354, 0.05053751, -0.008651152, 0.0092...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.08413525, 0.022250246, 0.032795943, -0.039...",part jack is stuck in davy jones locker while ...
2,206647,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[-0.08593852, 0.020284886, -0.0063891695, 0.02...","[-0.018606687, 0.059188254, -0.03516622, -0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.064327925, -0.035831474, -0.022186274, -0....",a hrefhttpwwwcutprintfilmcomreviewsspectrea in...
3,49026,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[0.0011890167, 0.046007097, -0.06765067, -0.01...","[-0.03928988, -0.015648525, -0.10462731, 0.042...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.004513586, -0.023706274, -0.052867, -0.023...",i felt like this was a tremendous end to nolan...
4,49529,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[-0.005292492, 0.031383008, 0.00044830245, -0....","[-0.02489186, -0.021951115, -0.014892352, 0.04...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.017391438, -0.037688196, 0.020004472, 0.02...",totally forgettable and full of stereotypes ed...


In [ ]:

finaldataset.to_pickle("finaldataset.pkl")

from google.colab import files
files.download("finaldataset.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>